# Deliverable 1: Automated Scenario Modeling System

**Objective:** Build a Monte Carlo simulation engine that projects operational KPIs under multiple scenarios (optimistic, baseline, pessimistic, worst-case) and uses Claude AI to interpret the results.

**What this notebook demonstrates:**
1. Synthetic data generation for 10 operational metrics across 3 manufacturing facilities
2. Statistical processing pipeline (trends, anomaly detection, correlations)
3. Monte Carlo scenario projections with configurable parameters
4. Interactive visualizations of projection uncertainty bands
5. AI-powered interpretation of scenario outcomes via Claude API

---

## Setup

In [ ]:
import sys
sys.path.insert(0, '../..')  # project root

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from shared.data_generation.generate import generate_operational_metrics
from shared.utils.processing import run_full_processing
from shared.utils.plotting import apply_portfolio_style, save_figure, COLORS, SCENARIO_COLORS
from shared.config.loader import SETTINGS

pd.set_option('display.max_columns', 20)
pd.set_option('display.float_format', '{:.2f}'.format)

print('Setup complete.')

## 1. Generate Synthetic Operational Data

We generate 24 months of data for 3 manufacturing plants, each tracking 10 KPIs.
The data includes realistic trends, seasonality, facility-level differences, and occasional anomalies.

In [ ]:
raw_df = generate_operational_metrics()

print(f'Dataset shape: {raw_df.shape}')
print(f'Date range: {raw_df["date"].min().date()} to {raw_df["date"].max().date()}')
print(f'Facilities: {raw_df["facility"].unique().tolist()}')
print(f'Metrics: {raw_df["metric"].nunique()}')
print()
raw_df.head(10)

In [ ]:
# Save raw data for other deliverables
from shared.data_generation.generate import save_data
save_data(raw_df, '../data')
save_data(raw_df, '../../02_insight_extraction/data')
save_data(raw_df, '../../03_strategic_dashboard/data')
print('Data saved to all deliverable folders.')

## 2. Exploratory Data Analysis

Before modeling, let's understand the data — distributions, trends, and facility differences.

In [ ]:
# Overview: metric distributions by facility
fig = px.box(
    raw_df, x='metric', y='value', color='facility',
    color_discrete_map=COLORS,
)
fig = apply_portfolio_style(fig, 'Metric Distributions by Facility')
fig.update_xaxes(tickangle=45)
fig.update_layout(height=500)
save_figure(fig, '../outputs/metric_distributions')
fig.show()

In [ ]:
# Time series: select key metrics
key_metrics = ['production_output', 'quality_rate', 'defect_rate_ppm', 'on_time_delivery_pct']

fig = px.line(
    raw_df[raw_df['metric'].isin(key_metrics)],
    x='date', y='value', color='facility', facet_row='metric',
    color_discrete_map=COLORS,
    height=800,
)
fig = apply_portfolio_style(fig, 'Key Metric Trends (24 Months)')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1].replace('_', ' ').title()))
save_figure(fig, '../outputs/key_metric_trends')
fig.show()

## 3. Data Processing Pipeline

Run statistical processing: summary stats, month-over-month trends, anomaly detection, and cross-metric correlations.

In [ ]:
processed = run_full_processing(raw_df)

for name, df in processed.items():
    print(f'{name}: {df.shape}')

print(f'\nAnomalies detected: {len(processed["anomalies"])}')

In [ ]:
# Summary stats
processed['summary'].head(15)

In [ ]:
# Visualize anomalies
anom = processed['anomalies']
if len(anom) > 0:
    fig = px.scatter(
        anom, x='date', y='z_score', color='facility', symbol='metric',
        size=anom['z_score'].abs(),
        color_discrete_map=COLORS,
    )
    fig.add_hline(y=2.0, line_dash='dash', line_color='red', opacity=0.5)
    fig.add_hline(y=-2.0, line_dash='dash', line_color='red', opacity=0.5)
    fig = apply_portfolio_style(fig, 'Detected Anomalies (|z-score| > 2.0)')
    fig.update_layout(height=450)
    save_figure(fig, '../outputs/anomalies')
    fig.show()
else:
    print('No anomalies detected.')

## 4. Monte Carlo Scenario Modeling

We project each metric forward 12 months under 4 scenarios:
- **Optimistic** (+15% trend amplification)
- **Baseline** (current trajectory)
- **Pessimistic** (-15% trend reversal)
- **Worst Case** (-30% trend reversal)

Each scenario runs 200 Monte Carlo simulations to capture uncertainty bands.

In [ ]:
import importlib, types

# Import from digit-prefixed folder using importlib
def import_from(module_path, module_name):
    """Import a module from a path that may start with digits."""
    import importlib.util
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

modeler = import_from('../../01_scenario_modeling/src/modeler.py', 'modeler')

scenarios = modeler.run_all_scenarios(raw_df)
flat_df = modeler.scenarios_to_flat_df(scenarios)
endpoint_df = modeler.scenarios_endpoint_summary(scenarios)

print(f'Scenario projections generated: {len(flat_df)} data points')
print(f'Endpoint summary: {len(endpoint_df)} rows')
endpoint_df.head(10)

In [ ]:
# Visualize scenario fan charts for a key metric
def plot_scenario_fan(scenarios_dict, facility, metric, title=None):
    """Plot scenario projection fan chart with uncertainty bands."""
    fig = go.Figure()
    
    for scenario_name, proj_df in scenarios_dict[facility][metric].items():
        color = SCENARIO_COLORS.get(scenario_name, '#888')
        
        # Uncertainty band (p10-p90)
        fig.add_trace(go.Scatter(
            x=pd.concat([proj_df['month'], proj_df['month'][::-1]]),
            y=pd.concat([proj_df['p90'], proj_df['p10'][::-1]]),
            fill='toself', fillcolor=color, opacity=0.15,
            line=dict(width=0), showlegend=False, name=f'{scenario_name} band',
        ))
        # Median line
        fig.add_trace(go.Scatter(
            x=proj_df['month'], y=proj_df['median'],
            mode='lines', name=scenario_name.replace('_', ' ').title(),
            line=dict(color=color, width=2.5),
        ))
    
    title = title or f'{metric.replace("_", " ").title()} — {facility} — 12 Month Projection'
    fig = apply_portfolio_style(fig, title)
    fig.update_xaxes(title_text='Months Ahead')
    fig.update_yaxes(title_text='Projected Value')
    fig.update_layout(height=450)
    return fig

fig = plot_scenario_fan(scenarios, 'Plant Alpha', 'production_output')
save_figure(fig, '../outputs/scenario_fan_production')
fig.show()

In [ ]:
fig = plot_scenario_fan(scenarios, 'Plant Beta', 'quality_rate')
save_figure(fig, '../outputs/scenario_fan_quality')
fig.show()

In [ ]:
fig = plot_scenario_fan(scenarios, 'Plant Gamma', 'defect_rate_ppm')
save_figure(fig, '../outputs/scenario_fan_defects')
fig.show()

In [ ]:
# Uncertainty range comparison across all metrics
uncertainty = endpoint_df.groupby(['metric', 'scenario'])['uncertainty_range'].mean().reset_index()

fig = px.bar(
    uncertainty, x='metric', y='uncertainty_range', color='scenario',
    barmode='group', color_discrete_map=SCENARIO_COLORS,
)
fig = apply_portfolio_style(fig, 'Projection Uncertainty by Metric & Scenario')
fig.update_xaxes(tickangle=45)
fig.update_layout(height=500)
save_figure(fig, '../outputs/uncertainty_comparison')
fig.show()

In [ ]:
# Facility resilience: how much does each facility's median drop under worst case vs baseline?
pivot = endpoint_df.pivot_table(
    index=['facility', 'metric'], columns='scenario', values='projected_median'
).reset_index()
pivot['resilience_pct'] = ((pivot['worst_case'] - pivot['baseline']) / pivot['baseline'] * 100).round(1)

fig = px.bar(
    pivot, x='metric', y='resilience_pct', color='facility',
    barmode='group', color_discrete_map=COLORS,
)
fig = apply_portfolio_style(fig, 'Facility Resilience: Worst Case vs Baseline (% Change)')
fig.update_xaxes(tickangle=45)
fig.update_layout(height=500)
save_figure(fig, '../outputs/facility_resilience')
fig.show()

## 5. AI-Powered Scenario Interpretation

We send the scenario modeling results to Claude API to generate strategic interpretation.

> **Note:** This section requires an `ANTHROPIC_API_KEY` in your `.env` file. If not set, the cells will show a placeholder message.

In [ ]:
try:
    analyst = import_from('../../01_scenario_modeling/src/scenario_analyst.py', 'scenario_analyst')
    
    analysis = analyst.analyze_scenario_results(endpoint_df.to_csv(index=False))
    print('=== SCENARIO ANALYSIS ===')
    print(analysis)
except Exception as e:
    print(f'AI analysis unavailable: {e}')
    print('Set ANTHROPIC_API_KEY in .env to enable AI-powered interpretation.')

In [ ]:
try:
    narrative = analyst.generate_scenario_narrative(endpoint_df.to_csv(index=False))
    print('=== EXECUTIVE BRIEFING ===')
    print(narrative)
except Exception as e:
    print(f'AI narrative unavailable: {e}')

## 6. Save Outputs

Export all scenario modeling artifacts for use in other deliverables.

In [ ]:
from pathlib import Path

output_dir = Path('../outputs')
output_dir.mkdir(exist_ok=True)

flat_df.to_csv(output_dir / 'scenario_projections.csv', index=False)
endpoint_df.to_csv(output_dir / 'scenario_endpoint_summary.csv', index=False)
processed['summary'].to_csv(output_dir / 'summary_stats.csv', index=False)
processed['anomalies'].to_csv(output_dir / 'anomalies.csv', index=False)
processed['correlations'].to_csv(output_dir / 'correlations.csv', index=False)

print('All outputs saved to 01_scenario_modeling/outputs/')
for f in sorted(output_dir.iterdir()):
    print(f'  {f.name}')

---

## Summary

This deliverable demonstrates:

| Capability | Implementation |
|---|---|
| Data Engineering | Synthetic data generator with realistic patterns, trends, and anomalies |
| Statistical Analysis | Summary stats, trend detection, z-score anomaly flagging, cross-metric correlations |
| Scenario Modeling | Monte Carlo simulation engine with configurable scenarios and uncertainty quantification |
| Data Visualization | Interactive Plotly fan charts, uncertainty bars, resilience comparisons |
| AI Integration | Claude API for strategic scenario interpretation and executive briefing generation |

**Next:** See [02_insight_extraction](../../02_insight_extraction/notebooks/) for the AI insight extraction framework.